## Imports

In [1]:
import itertools

from collections import namedtuple
from dataclasses import dataclass
from functools import wraps
from typing import Callable, Set

import numpy as np

from aoc_utilities import puzzle_input

<h2>--- Day 8: Seven Segment Search ---</h2><p>You barely reach the safety of the cave when the whale smashes into the cave mouth, collapsing it. Sensors indicate another exit to this cave at a much greater depth, so you have no choice but to press on.</p>
<p>As your submarine slowly makes its way through the cave system, you notice that the four-digit <a href="https://en.wikipedia.org/wiki/Seven-segment_display" target="_blank">seven-segment displays</a> in your submarine are malfunctioning; <span title="Yes, just the four-digit seven-segment ones. Whole batch must have been faulty.">they must have been damaged</span> during the escape. You'll be in a lot of trouble without them, so you'd better figure out what's wrong.</p>
<p>Each digit of a seven-segment display is rendered by turning on or off any of seven segments named <code>a</code> through <code>g</code>:</p>
<pre><code>  0:      1:      2:      3:      4:
 <em>aaaa</em>    ....    <em>aaaa    aaaa</em>    ....
<em>b    c</em>  .    <em>c</em>  .    <em>c</em>  .    <em>c  b    c</em>
<em>b    c</em>  .    <em>c</em>  .    <em>c</em>  .    <em>c  b    c</em>
 ....    ....    <em>dddd    dddd    dddd</em>
<em>e    f</em>  .    <em>f  e</em>    .  .    <em>f</em>  .    <em>f</em>
<em>e    f</em>  .    <em>f  e</em>    .  .    <em>f</em>  .    <em>f</em>
 <em>gggg</em>    ....    <em>gggg    gggg</em>    ....

  5:      6:      7:      8:      9:
 <em>aaaa    aaaa    aaaa    aaaa    aaaa</em>
<em>b</em>    .  <em>b</em>    .  .    <em>c  b    c  b    c</em>
<em>b</em>    .  <em>b</em>    .  .    <em>c  b    c  b    c</em>
 <em>dddd    dddd</em>    ....    <em>dddd    dddd</em>
.    <em>f  e    f</em>  .    <em>f  e    f</em>  .    <em>f</em>
.    <em>f  e    f</em>  .    <em>f  e    f</em>  .    <em>f</em>
 <em>gggg    gggg</em>    ....    <em>gggg    gggg</em>
</code></pre>
<p>So, to render a <code>1</code>, only segments <code>c</code> and <code>f</code> would be turned on; the rest would be off. To render a <code>7</code>, only segments <code>a</code>, <code>c</code>, and <code>f</code> would be turned on.</p>
<p>The problem is that the signals which control the segments have been mixed up on each display. The submarine is still trying to display numbers by producing output on signal wires <code>a</code> through <code>g</code>, but those wires are connected to segments <em>randomly</em>. Worse, the wire/segment connections are mixed up separately for each four-digit display! (All of the digits <em>within</em> a display use the same connections, though.)</p>
<p>So, you might know that only signal wires <code>b</code> and <code>g</code> are turned on, but that doesn't mean <em>segments</em> <code>b</code> and <code>g</code> are turned on: the only digit that uses two segments is <code>1</code>, so it must mean segments <code>c</code> and <code>f</code> are meant to be on. With just that information, you still can't tell which wire (<code>b</code>/<code>g</code>) goes to which segment (<code>c</code>/<code>f</code>). For that, you'll need to collect more information.</p>
<p>For each display, you watch the changing signals for a while, make a note of <em>all ten unique signal patterns</em> you see, and then write down a single <em>four digit output value</em> (your puzzle input). Using the signal patterns, you should be able to work out which pattern corresponds to which digit.</p>
<p>For example, here is what you might see in a single entry in your notes:</p>
<pre><code>acedgfb cdfbe gcdfa fbcad dab cefabd cdfgeb eafb cagedb ab |
cdfeb fcadb cdfeb cdbaf</code></pre>
<p>(The entry is wrapped here to two lines so it fits; in your notes, it will all be on a single line.)</p>
<p>Each entry consists of ten <em>unique signal patterns</em>, a <code>|</code> delimiter, and finally the <em>four digit output value</em>. Within an entry, the same wire/segment connections are used (but you don't know what the connections actually are). The unique signal patterns correspond to the ten different ways the submarine tries to render a digit using the current wire/segment connections. Because <code>7</code> is the only digit that uses three segments, <code>dab</code> in the above example means that to render a <code>7</code>, signal lines <code>d</code>, <code>a</code>, and <code>b</code> are on. Because <code>4</code> is the only digit that uses four segments, <code>eafb</code> means that to render a <code>4</code>, signal lines <code>e</code>, <code>a</code>, <code>f</code>, and <code>b</code> are on.</p>
<p>Using this information, you should be able to work out which combination of signal wires corresponds to each of the ten digits. Then, you can decode the four digit output value. Unfortunately, in the above example, all of the digits in the output value (<code>cdfeb fcadb cdfeb cdbaf</code>) use five segments and are more difficult to deduce.</p>
<p>For now, <em>focus on the easy digits</em>. Consider this larger example:</p>
<pre><code>be cfbegad cbdgef fgaecd cgeb fdcge agebfd fecdb fabcd edb |
<em>fdgacbe</em> cefdb cefbgd <em>gcbe</em>
edbfga begcd cbg gc gcadebf fbgde acbgfd abcde gfcbed gfec |
fcgedb <em>cgb</em> <em>dgebacf</em> <em>gc</em>
fgaebd cg bdaec gdafb agbcfd gdcbef bgcad gfac gcb cdgabef |
<em>cg</em> <em>cg</em> fdcagb <em>cbg</em>
fbegcd cbd adcefb dageb afcb bc aefdc ecdab fgdeca fcdbega |
efabcd cedba gadfec <em>cb</em>
aecbfdg fbg gf bafeg dbefa fcge gcbea fcaegb dgceab fcbdga |
<em>gecf</em> <em>egdcabf</em> <em>bgf</em> bfgea
fgeab ca afcebg bdacfeg cfaedg gcfdb baec bfadeg bafgc acf |
<em>gebdcfa</em> <em>ecba</em> <em>ca</em> <em>fadegcb</em>
dbcfg fgd bdegcaf fgec aegbdf ecdfab fbedc dacgb gdcebf gf |
<em>cefg</em> dcbef <em>fcge</em> <em>gbcadfe</em>
bdfegc cbegaf gecbf dfcage bdacg ed bedf ced adcbefg gebcd |
<em>ed</em> bcgafe cdgba cbgef
egadfb cdbfeg cegd fecab cgb gbdefca cg fgcdab egfdb bfceg |
<em>gbdfcae</em> <em>bgc</em> <em>cg</em> <em>cgb</em>
gcafb gcf dcaebfg ecagb gf abcdeg gaef cafbge fdbac fegbdc |
<em>fgae</em> cfgab <em>fg</em> bagce
</code></pre>
<p>Because the digits <code>1</code>, <code>4</code>, <code>7</code>, and <code>8</code> each use a unique number of segments, you should be able to tell which combinations of signals correspond to those digits. Counting <em>only digits in the output values</em> (the part after <code>|</code> on each line), in the above example, there are <code><em>26</em></code> instances of digits that use a unique number of segments (highlighted above).</p>
<p><em>In the output values, how many times do digits <code>1</code>, <code>4</code>, <code>7</code>, or <code>8</code> appear?</em></p>


## Solution

In [2]:
@dataclass(frozen=True)
class DisplayIO:
    inputs: list[set[str]]
    outputs: list[set[str]]

    @classmethod
    def from_note(cls, note):
        signals = [set(signal) for signal in note.split()]
        return cls(inputs=signals[:10], outputs=signals[11:])        


def parse_seven_segment_display_notes(notes: str):
    return map(DisplayIO.from_note, notes.split('\n'))


def count_output_digits_1_4_7_and_8(display_io: DisplayIO):
    allowed_segment_counts = {2, 3, 4, 7}
    return sum(
        len(signal_pattern) in allowed_segment_counts
        for signal_pattern in display_io.outputs
    )


print(
    'Number of 1, 4, 7, and 8 digits in display outputs:',
    sum(
        count_output_digits_1_4_7_and_8(display_io)
        for display_io in parse_seven_segment_display_notes(puzzle_input.as_text(day=8))
    )
)

Number of 1, 4, 7, and 8 digits in display outputs: 301


<h2 id="part2">--- Part Two ---</h2><p>Through a little deduction, you should now be able to determine the remaining digits. Consider again the first example above:</p>
<pre><code>acedgfb cdfbe gcdfa fbcad dab cefabd cdfgeb eafb cagedb ab |
cdfeb fcadb cdfeb cdbaf</code></pre>
<p>After some careful analysis, the mapping between signal wires and segments only make sense in the following configuration:</p>
<pre><code> dddd
e    a
e    a
 ffff
g    b
g    b
 cccc
</code></pre>
<p>So, the unique signal patterns would correspond to the following digits:</p>
<ul>
<li><code>acedgfb</code>: <code>8</code></li>
<li><code>cdfbe</code>: <code>5</code></li>
<li><code>gcdfa</code>: <code>2</code></li>
<li><code>fbcad</code>: <code>3</code></li>
<li><code>dab</code>: <code>7</code></li>
<li><code>cefabd</code>: <code>9</code></li>
<li><code>cdfgeb</code>: <code>6</code></li>
<li><code>eafb</code>: <code>4</code></li>
<li><code>cagedb</code>: <code>0</code></li>
<li><code>ab</code>: <code>1</code></li>
</ul>
<p>Then, the four digits of the output value can be decoded:</p>
<ul>
<li><code>cdfeb</code>: <code><em>5</em></code></li>
<li><code>fcadb</code>: <code><em>3</em></code></li>
<li><code>cdfeb</code>: <code><em>5</em></code></li>
<li><code>cdbaf</code>: <code><em>3</em></code></li>
</ul>
<p>Therefore, the output value for this entry is <code><em>5353</em></code>.</p>
<p>Following this same process for each entry in the second, larger example above, the output value of each entry can be determined:</p>
<ul>
<li><code>fdgacbe cefdb cefbgd gcbe</code>: <code>8394</code></li>
<li><code>fcgedb cgb dgebacf gc</code>: <code>9781</code></li>
<li><code>cg cg fdcagb cbg</code>: <code>1197</code></li>
<li><code>efabcd cedba gadfec cb</code>: <code>9361</code></li>
<li><code>gecf egdcabf bgf bfgea</code>: <code>4873</code></li>
<li><code>gebdcfa ecba ca fadegcb</code>: <code>8418</code></li>
<li><code>cefg dcbef fcge gbcadfe</code>: <code>4548</code></li>
<li><code>ed bcgafe cdgba cbgef</code>: <code>1625</code></li>
<li><code>gbdfcae bgc cg cgb</code>: <code>8717</code></li>
<li><code>fgae cfgab fg bagce</code>: <code>4315</code></li>
</ul>
<p>Adding all of the output values in this larger example produces <code><em>61229</em></code>.</p>
<p>For each entry, determine all of the wire/segment connections and decode the four-digit output values. <em>What do you get if you add up all of the output values?</em></p>


## Solution

In [3]:
def decode_output_signal(display_io: DisplayIO):
    digit_signals = {
        1: find_input_by_segment_count(display_io, 2),
        4: find_input_by_segment_count(display_io, 4),
        7: find_input_by_segment_count(display_io, 3),
        8: find_input_by_segment_count(display_io, 7)
    }

    signals_by_length = {
        k: list(map(set, g))
        for k, g in itertools.groupby(sorted(display_io.inputs, key=len), len)
    }

    digit_signals[0] = find_0(signals_by_length)
    digit_signals[2] = find_2_given_4(digit_signals, signals_by_length)
    digit_signals[3] = find_3_given_2(digit_signals, signals_by_length)
    digit_signals[5] = find_5_given_2(digit_signals, signals_by_length)
    digit_signals[6] = find_6_given_0_and_7(digit_signals, signals_by_length)
    digit_signals[9] = find_9_given_0_and_7(digit_signals, signals_by_length)

    signal_digit = {
        frozenset(signal): digit
        for digit, signal in digit_signals.items()
    }

    return int(
        ''.join(
            str(signal_digit[frozenset(signal)])
            for signal in display_io.outputs
        )
    )


def find_input_by_segment_count(display_io: DisplayIO, count):
    inputs = [
        input_signal
        for input_signal in display_io.inputs
        if len(input_signal) == count
    ]
    assert len(inputs) == 1
    return set(inputs[0])


def unwrap_output(f: Callable[..., Set]):
    @wraps(f)
    def f_unwrapped_output(*args, **kwargs):
        f_output = f(*args, **kwargs)
        assert len(f_output) == 1
        return f_output.copy().pop()
    return f_unwrapped_output


@unwrap_output
def find_0(signals_by_length):
    return [
        six_segment_digit
        for six_segment_digit in signals_by_length[6]
        if all(
            len(five_segment_digit - six_segment_digit) == 1
            for five_segment_digit in signals_by_length[5]
        )
    ]


@unwrap_output
def find_2_given_4(digit_signals, signals_by_length):
    return [
        five_segment
        for five_segment in signals_by_length[5]
        if len(five_segment - digit_signals[4]) == 3
    ]


@unwrap_output
def find_3_given_2(digit_signals, signals_by_length):
    return [
        five_segment
        for five_segment in signals_by_length[5]
        if len(five_segment - digit_signals[2]) == 1
    ]


@unwrap_output
def find_5_given_2(digit_signals, signals_by_length):
    return [
        five_segment
        for five_segment in signals_by_length[5]
        if len(five_segment - digit_signals[2]) == 2
    ]


@unwrap_output
def find_6_given_0_and_7(digit_signals, signals_by_length):
    return [
        six_segment
        for six_segment in signals_by_length[6]
        if six_segment != digit_signals[0]
            and len(six_segment - digit_signals[7]) == 4
    ]


@unwrap_output
def find_9_given_0_and_7(digit_signals, signals_by_length):
    return [
        six_segment
        for six_segment in signals_by_length[6]
        if six_segment != digit_signals[0]
            and len(six_segment - digit_signals[7]) == 3
    ]

### Basic Testing

In [4]:
simple_example = next(
    parse_seven_segment_display_notes(
        'acedgfb cdfbe gcdfa fbcad dab cefabd cdfgeb eafb cagedb ab'
        ' | cdfeb fcadb cdfeb cdbaf'
    )
)


assert decode_output_signal(simple_example) == 5353

### Answer

In [5]:
print(
    'Sum of all output values:',
    sum(
        int(decode_output_signal(display_io))
        for display_io in parse_seven_segment_display_notes(puzzle_input.as_text(day=8))
    )
)

Sum of all output values: 908067
